# Using btrack with napari


In [1]:
import btrack
import napari

In [2]:
print(btrack.__version__)

0.4.3


In [3]:
objects = btrack.dataio.import_CSV('napari_example.csv')
# objects = btrack.dataio.import_CSV('../tests/_test_data/test_data.csv')

In [4]:
objects[0]

,ID,x,y,z,t,dummy,states,label,prob
0,0,517.559509,9.081633,0.0,0,False,0,0,0.0


In [5]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file('../models/cell_config.json')

    # append the objects to be tracked
    tracker.append(objects)

    # set the volume (Z axis volume is set very large for 2D data)
    tracker.volume=((0,1600), (0,1200), (-1e5,1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

#     tracker.export('./test3.hdf5', obj_type='obj_type_1')

    # get the tracks in a format for napari visualization
    data, properties, graph = tracker.to_napari(ndim=2)

[INFO][2022/02/17 09:37:34 AM] Loaded btrack: /Users/arl/Dropbox/Code/py3/BayesianTracker/btrack/libs/libtracker.dylib
[WARNING][2022/02/17 09:37:34 AM] btrack (v0.4.3) shared library mismatch.
[INFO][2022/02/17 09:37:34 AM] Setting max XYZ search radius to: 100
[INFO][2022/02/17 09:37:34 AM] Starting BayesianTracker session
[INFO][2022/02/17 09:37:34 AM] Loading configuration file: ../models/cell_config.json
[INFO][2022/02/17 09:37:34 AM] Loading motion model: cell_motion
[INFO][2022/02/17 09:37:34 AM] Objects are of type: <class 'list'>
[INFO][2022/02/17 09:37:34 AM] Set volume to ((0, 1600), (0, 1200), (-100000.0, 100000.0))
[INFO][2022/02/17 09:37:34 AM] Starting tracking... 
[INFO][2022/02/17 09:37:34 AM] Tracking objects in frames 0 to 99 (of 500)...
[INFO][2022/02/17 09:37:34 AM]  - Timing (Bayesian updates: 0.94ms, Linking: 0.16ms)
[INFO][2022/02/17 09:37:34 AM]  - Probabilities (Link: 1.00000, Lost: 0.49482)
[INFO][2022/02/17 09:37:34 AM]  - Stats (Active: 78, Lost: 194, Confl

Instantiating BTRACK interface wrapper (v0.4.3, compiled Nov 25 2021 at 11:31:42)


[INFO][2022/02/17 09:37:35 AM]  - Timing (Bayesian updates: 1.11ms, Linking: 0.17ms)
[INFO][2022/02/17 09:37:35 AM]  - Probabilities (Link: 1.00000, Lost: 0.60258)
[INFO][2022/02/17 09:37:35 AM]  - Stats (Active: 83, Lost: 326, Conflicts resolved: 187)
[INFO][2022/02/17 09:37:35 AM] Tracking objects in frames 200 to 299 (of 500)...
[INFO][2022/02/17 09:37:35 AM]  - Timing (Bayesian updates: 2.09ms, Linking: 0.25ms)
[INFO][2022/02/17 09:37:35 AM]  - Probabilities (Link: 1.00000, Lost: 0.27376)
[INFO][2022/02/17 09:37:35 AM]  - Stats (Active: 113, Lost: 574, Conflicts resolved: 276)
[INFO][2022/02/17 09:37:35 AM] Tracking objects in frames 300 to 399 (of 500)...
[INFO][2022/02/17 09:37:35 AM]  - Timing (Bayesian updates: 3.38ms, Linking: 0.32ms)
[INFO][2022/02/17 09:37:35 AM]  - Probabilities (Link: 0.99997, Lost: 0.99982)
[INFO][2022/02/17 09:37:35 AM]  - Stats (Active: 140, Lost: 1001, Conflicts resolved: 401)
[INFO][2022/02/17 09:37:35 AM] Tracking objects in frames 400 to 499 (of 500

GLPK Integer Optimizer, v4.65
2028 rows, 1992 columns, 3089 non-zeros
1992 integer variables, all of which are binary
Preprocessing...
1014 rows, 1992 columns, 3089 non-zeros
1992 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1014
Solving LP relaxation...
GLPK Simplex Optimizer, v4.65
1014 rows, 1992 columns, 3089 non-zeros
*     0: obj =   4.053900533e+03 inf =   0.000e+00 (365)
*   343: obj =   8.200734509e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+   343: mip =     not found yet >=              -inf        (1; 0)
+   343: >>>>>   8.200734509e+02 >=   8.200734509e+02   0.0% (1; 0)
+   343: mip =   8.200734509e+02 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUND
Deleting BTRACK interface wrapper


In [6]:
# with napari.gui_qt():
viewer = napari.Viewer()
viewer.add_tracks(data, properties=properties, graph=graph, name='tracks')

<Tracks layer 'tracks' at 0x142fe2a60>

In [8]:
# graph